In [1]:
import pandas as pd
import re

In [3]:
tweets = pd.read_csv('LabeledTweets.csv', names=['sentiment', 'id', 'timestamp', 'query', 'user', 'text'])
tweets.head()

,sentiment,id,timestamp,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


C:\Users\taillow40\AppData\Local\Temp\ipykernel_220572\2039518796.py:1: DtypeWarning: Columns (2,3,4,6,9,12,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  foods = pd.read_csv("../FoodData/branded_food.csv")


,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,household_serving_fulltext,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
0,1105904,Richardson Oilseed Products (US) Limited,NaN,NaN,27000612323,Vegetable Oil,NaN,15.0,ml,NaN,Oils Edible,GDSN,NaN,2020-10-02,2020-11-13,United States,NaN,NaN,NaN,NaN
1,1105905,CAMPBELL SOUP COMPANY,NaN,NaN,51000198808,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,240.0,ml,NaN,Herbs/Spices/Extracts,GDSN,NaN,2020-09-12,2020-11-13,United States,NaN,NaN,NaN,NaN
2,1105906,CAMPBELL SOUP COMPANY,NaN,NaN,51000213273,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,440.0,g,NaN,Prepared Soups,GDSN,NaN,2020-09-01,2020-11-13,United States,NaN,NaN,NaN,NaN
3,1105907,CAMPBELL SOUP COMPANY,NaN,NaN,51000213303,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,440.0,g,NaN,Prepared Soups,GDSN,NaN,2020-09-01,2020-11-13,United States,NaN,NaN,NaN,NaN
4,1105908,CAMPBELL SOUP COMPANY,NaN,NaN,51000224637,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,240.0,ml,NaN,Herbs/Spices/Extracts,GDSN,NaN,2020-10-03,2020-11-13,United States,NaN,NaN,NaN,NaN


In [26]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Initialize tools
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
def preprocess_tweet(tweet):
    oldTweet = tweet
    # 1. Remove non-alphanumeric characters
    tweet = re.sub(r"[^a-zA-Z0-9\s']", "", tweet) 

    # 2. Convert to lowercase
    tweet = tweet.lower()

    # 3. Expand contractions
    contractions = {
        "isn't": "is not", "aren't": "are not", "wasn't": "was not", "weren't": "were not",
        "can't": "cannot", "couldn't": "could not", "shouldn't": "should not", "won't": "will not",
        "wouldn't": "would not", "didn't": "did not", "hasn't": "has not", "haven't": "have not",
        "doesn't": "does not", "don't": "do not", "i'm": "i am", "it's": "it is", "you're": "you are",
        "they're": "they are", "we're": "we are", "i've": "i have", "you've": "you have"
    }
    for contraction, expanded in contractions.items():
        tweet = tweet.replace(contraction, expanded)
    tweet = re.sub(r"[']", "", tweet)

    # 4. Handle negations
    words = tweet.split(" ")
    processed_words = []
    negate = False
    for word in words:
        if word in ["not", "no", "never"]:
            negate = True
        elif negate and word not in stop_words:
            processed_words.append(f"not_{word}")
            negate = False
        else:
            processed_words.append(word)

    # 5. Remove stopwords
    filtered_words = [word for word in processed_words if word not in stop_words]

    # 6. Lemmatize words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    new_tweet = " ".join(lemmatized_words)
    new_tweet = re.sub(r'\s+', ' ', new_tweet)
    return new_tweet

# Preprocess the dataset
processed_tweets = [preprocess_tweet(tweet) for tweet in tweets['text']]


In [58]:
def preprocess(text):
    text = text.lower()            
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\W+', ' ', text)    
    return text.strip()

foods = pd.read_csv("../FoodData/input_food.csv")
foods['sr_description'] = foods['sr_description'].apply(preprocess)
words = set(word for desc in foods['sr_description'].dropna() for word in desc.split())

# Remove common words from the set 
common_words = {"the", "and", "no", "to", "be", "or", "a", "go", "year", "off", 
                "good", "up", "home", "mixed", "been", "great", "valley", 
                "choice", "red", "sun", "dog", "all", "of", "food", "with", "dark", "part", "for", "not", "w", "hard", "people", "as", "old", "like", "in", "fast", "butt", "only", "low", "white", "eat", "at", "hot",
                "o", "s", "e", "b", "at", "on", "made", "pieces", "bowl", "drinking", "such", "stomach", "new", "have", "cut", "ready", "jack", "green", "yellow", "back", "snap", "dry", "brown", "way", "table", "replacement", "small", "high", "lil", "from", "may", "special", "summer", "next", "cam", "back", "acting", "active", "additional", "advance", "agents", "air", "aluminum", "approximately", "areas", "arm", "average", "balance", "basis", "blade", "boost", "brain", "charged", "classes", "club", "combos", "common", "company", "compressed", "contents", "cytosport", "denver", "diabetes", "diarrhea", "dogs", "eastern", "except", "expert", "family", "feed", "fiber", "flesh", "fluid", "fordhook", "form", "formed", "formula", "general", "greater", "group", "inside", "iron", "island", "machine", "mature", "method", "mild", "mineral", "minerals", "multi", "muscle", "municipal", "other", "paper", "performance", "previously", "prime", "processing", "product", "products", "program", "purpose", "regular", "removed", "restaurant", "revive", "sensitive", "single", "size", "slightly", "solution", "standard", "straight", "style", "type", "types", "typical", "ultra", "under", "use", "varieties", "variety", "wild", "without", "zone", "abbott", "added", "additives", "american", "amp", "animal", "any", "ar", "ara", "armenian", "artificial", "assorted", "babyfood", "beach", "bear", "beaver", "bird", "bitter", "black", "bleached", "block", "blood", "boxed", "bulk", "bull", "bunches", "by", "c", "ca", "calcium", "calorie", "can", "candied", "carton", "center", "chain", "channel", "child", "classic", "clif", "commercial", "commercially", "composite", "con", "containing", "contains", "cottonseed", "country", "cracked", "crude", "cultured", "d", "dha", "dietary", "dietetic", "distilled", "domesticated", "eas", "electrolyte", "end", "energy", "english", "equal", "european", "extra", "extract", "eye", "fashioned", "fed", "fine", "finger", "fluoride", "fortified", "free", "fresh", "full", "fully", "g", "game", "generic", "gentle", "genuine", "greater", "grow", "gummy", "half", "headcheese", "heart", "hearts", "higher", "household", "hydrated", "immature", "industrial", "infant", "instant", "iron", "isolate", "johnson", "junior", "kit", "king", "ladyfingers", "large", "latino", "leaf", "leather", "leaves", "less", "light", "lightly", "link", "links", "lip", "liquid", "liquids", "loaded", "lotus", "lower", "m", "maid", "mature", "menu", "method", "mild", "mills", "mini", "minute", "mixture", "moisture", "monster", "mum", "multi", "musketeers", "native", "natural", "nature", "navels", "non", "nos", "ns", "optima", "other", "pack", "packaged", "packet", "packets", "perfect", "performance", "plus", "pm", "portion", "purchased", "purpose", "reduced", "refrigerated", "removed", "restaurant", "retail", "retain", "rich", "rockstar", "rush", "schiff", "school", "alaska", "amber", "asian", "atlantic", "baby", "baker", "bakery", "base", "based", "bell", "bengal", "blend", "blended", "blends", "blue", "brick", "boston", "brittle", "bulb", "button", "care", "carp", "celestial", "cellophane", "chinese", "chloride", "cholesterol", "clod", "colored", "concentrate", "contain", "covered", "crumbs", "crust", "distribution", "double", "end", "expressed", "extruded", "farmed", "feet", "filled", "fillings", "flowers", "fortune", "gatorade", "generic", "gentlease", "gerber", "goddess", "gold", "graduates", "grain", "gram", "granular", "granules", "grass", "grecian", "greater", "group", "anhydrous", "balsam", "de", "decker", "decorticated", "defatted", "degermed", "del", "drained", "extruded", "gelatin", "germ", "grade", "grades", "neutral", "packed", "plan", "preserving", "sections", "solid", "solids", "stable", "stage", "tabletop", "tap", "total", "transitions", "trimmed", "unblanched", "unbleached", "uncoated", "uncooked", "uncreamed", "undiluted", "unenriched", "unheated", "unprepared", "unspecified", "unsugared", "vault", "zero", "start", "long", "dinner", "breakfast", "drink", "top", "usda", "vacuum", "flavor", "passion", "sweet", "whole", "lunch", "garden", "spring", "round", "bar", "south", "power", "marathon", "pink", "quick", "original", "rose", "simple", "medium", "joe", "delicious", "ground", "stick", "drop", "recipe", "process", "wrap", "pre", "includes", "semi", "mix", "proof", "shelf", "spit", "smooth", "slim", "lean", "toddler", "iced", "traditional", "serve", "prepared", "raw", "packaging", "nutrition", "ingredients", "stir", "plain", "ingredient", "cold", "short", "round", "french", "winter", "cooking", "diet", "florida", "pop", "heat", "bite", "purple", "trail", "include", "oven", "plate", "flat", "eaten", "split", "us", "pad", "squirrel", "heavy", "z", "smith", "granny", "raccoon", "picnic", "soft", "prop", "globe", "hash", "toaster", "sticker", "lite", "vera", "luncheon", "yung", "chopped", "condensed"}

words = words - common_words

print(words)

{'flakes', 'enriched', 'alfredo', 'pomegranates', 'meatballs', 'fava', 'marmalades', 'sake', 'knackwurst', 'throttle', 'parsley', 'pearl', 'krispies', 'oranges', 'crumbled', 'tso', 'muffin', 'ramen', 'loaf', 'ostrich', 'bartlett', 'cranberry', 'semisoft', 'patties', 'kiwifruit', 'cookies', 'granola', 'asparagus', 'stews', 'falafel', 'shank', 'soyburgers', 'alcoholic', 'veal', 'mayo', 'ice', 'raisins', 'herbal', 'onions', 'roasted', 'similac', 'clam', 'recipes', 'rabbit', 'egg', 'turkey', 'creme', 'crayfish', 'seco', 'chard', 'marmalade', 'mackerel', 'grease', 'pudding', 'condiments', 'unsalted', 'oils', 'nachos', 'sorbitol', 'italian', 'creamed', 'reconstituted', 'mortadella', 'agave', 'papaya', 'orange', 'caesar', 'acai', 'prepackaged', 'evaporated', 'laver', 'coriander', 'treats', 'alcohol', 'fresco', 'flaked', 'crisps', 'feta', 'snapper', 'caramel', 'mushrooms', 'curry', 'lard', 'doughnut', 'peas', 'shake', 'payday', 'aloe', 'marinara', 'waffle', 'roma', 'sticks', 'spinach', 'flavor

In [59]:
sorted_words = sorted(list(words))
pd.DataFrame({'words': sorted_words}).to_csv('food_words.csv', index=False)


In [60]:
#tweets is a df with a column text that contains the tweets
#words is a set of words
#sort the tweets by the number of words in the tweet that are in the set words
#print them, only the text column and how many of the words match in the format {words}:{text}
def count_matching_words(text, words_set):
    return sum(1 for word in text.split() if word in words_set)

def highlight_matching_words(text, words_set):
    return ' '.join([f"{word}*" if word in words_set else word for word in text.split()])

output_counts = [count_matching_words(text, words) for text in processed_tweets]



In [61]:
#make a df of output_count and preprocessed_tweets
#drop the rows with 0 output_count
#save to csv
data = {'output_count': output_counts, 'preprocessed_tweets': processed_tweets}
df = pd.DataFrame(data)
filtered_df = df[df['output_count'] != 0]
filtered_df.to_csv('filtered_tweets.csv', index=False)